In [ ]:
from transformers import  utils, Trainer, TrainingArguments, ElectraTokenizer, ElectraForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import torch
from accelerate import Accelerator
import os
from performance import PerformanceSaver
from bertviz import model_view
utils.logging.set_verbosity_error()

In [ ]:
LOAD_SAVED_MODEL = False
saved_model_path = "models/binary/electra_classifier"
model_name = "howey/electra-base-mnli"
data_dir = "data/binary"

In [ ]:
if LOAD_SAVED_MODEL:
    tokenizer = ElectraTokenizer.from_pretrained(saved_model_path)
    model = ElectraForSequenceClassification.from_pretrained(saved_model_path, num_labels=2, ignore_mismatched_sizes=True, classifier_dropout=0.1, output_attentions=True)
else:
    tokenizer = ElectraTokenizer.from_pretrained(model_name)
    model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True, classifier_dropout=0.1)

if torch.cuda.is_available():
    model = model.cuda()


In [ ]:
training_args = TrainingArguments(
            output_dir='./results',
            do_eval=True,
            do_train=True,
            num_train_epochs=6,
            save_total_limit=2,
            load_best_model_at_end=True,
            learning_rate=8.5e-05,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            save_strategy="steps",
            logging_strategy="steps",
            evaluation_strategy="steps",
            logging_steps=200,
            eval_steps=200,
            save_steps=200,
        )

def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(
                labels, preds, average="weighted"
            )
            acc = accuracy_score(labels, preds)
            return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
dev_df = pd.read_csv(os.path.join(data_dir, 'dev.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
print(label_encoder.classes_)
train_df['label'] = label_encoder.transform(train_df['label'])
dev_df['label'] = label_encoder.transform(dev_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])


dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

def process(batch):
    inputs = tokenizer(batch["text"], truncation=True, padding="max_length")
    return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": batch["label"],
        }
    
tokenized_dataset = dataset.map(process, batched=True, remove_columns=dataset["train"].column_names)

In [ ]:
trainer = Trainer(
            model=model,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            tokenizer=tokenizer,
        )

In [ ]:
if not LOAD_SAVED_MODEL:
    trainer.train()
    trainer.save_model("models/binary/electra_classifier")
    predictions = trainer.predict(tokenized_dataset["test"])
    print(predictions.metrics)

## Inference

In [ ]:
device = torch.device("cuda:0")  # or whichever GPU you want to use
model = model.to(device)

In [ ]:
def infer(data, return_attention=False):
    tokenized_dataset = tokenizer(data, padding=True, return_tensors='pt')
    tokenized_dataset = tokenized_dataset.to(device)  
    #tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
    model.eval()
    with torch.no_grad():
        outputs = model(**tokenized_dataset)
    predicted_class_scores = outputs.logits
    return torch.argmax(predicted_class_scores, dim=1)

## Test set

In [ ]:

predicted_classes = infer(list(test_df['text'].values)).cpu()
true_labels = test_df['label'].values

accuracy = accuracy_score(true_labels, predicted_classes)
precision = precision_score(true_labels, predicted_classes, average='weighted')
recall = recall_score(true_labels, predicted_classes, average='weighted')
f1 = f1_score(true_labels, predicted_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

## Climate

In [ ]:
df_climate = pd.read_csv('data/climate_test.csv')
df_climate = df_climate[['text', 'label']]
df_climate['label'] = 1

predicted_classes = infer(list(df_climate['text'].values)).cpu()
true_labels = [1] * len(predicted_classes)

accuracy = accuracy_score(true_labels, predicted_classes)
precision = precision_score(true_labels, predicted_classes, average='weighted')
recall = recall_score(true_labels, predicted_classes, average='weighted')
f1 = f1_score(true_labels, predicted_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

## False Claims?

How does this model classify false claims??

In [ ]:
claims = pd.read_json('data/watclaimcheck data/train.json')

In [ ]:
def retrieve_claims(original_df, claim_type):
    text = original_df[original_df.apply(lambda x: 'original_rating' in x['label'] and x['label']['original_rating'] == claim_type, axis=1)]
    text['metadata'] = text['metadata'].apply(lambda x: x['claim'])
    return list(text['metadata'].values)

#### False claims

In [ ]:
false_claims = retrieve_claims(claims, 'false')
preds = infer(false_claims)
len(preds[preds==1])

#### Pants-fire claims

In [ ]:
pants_fire = retrieve_claims(claims, 'pants-fire')
preds = infer(pants_fire)
len(preds[preds==1])

In [ ]:
your_sentence = "You need to go to the party with me, otherwise you’ll just be bored at home."
from bertviz import head_view
def inference(text, classes):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt").to(model.device)
    model.eval( )
    with torch.no_grad():
        outputs = model(input_ids)
        attention = outputs[-1]
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        model_view(attention, tokens)
        logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return classes[predicted_class]

inference(your_sentence, label_encoder.classes_)

In [ ]:
#3del model
#del trainer
#torch.cuda.empty_cache()